# Serving ML Predictions in batch and real-time

**Learning objectives**

1. Copy trained model into your bucket
2. Deploy AI Platform trained model

## Introduction

In this notebook, we will create a prediction service that calls your trained model deployed in Cloud to serve predictions.

## Copy trained model

Set necessary variables

In [ ]:
PROJECT = "cloud-training-demos"
BUCKET = PROJECT
REGION = "us-central1"
TFVERSION = "2.1"

In [ ]:
import os
os.environ["BUCKET"] = BUCKET
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = TFVERSION

Create a bucket and copy trained model.

In [ ]:
%%bash
if ! gsutil ls -r gs://${BUCKET} | grep -q gs://${BUCKET}/babyweight/trained_model/; then
    gsutil mb -l ${REGION} gs://${BUCKET}
    gsutil -m cp -R gs://cloud-training-demos/babyweight/trained_model g://${BUCKET}/babyweight
fi

## Deploy trained model

We'll now deploy our model. This will take a few minutes. Once the cell below completes, you should be able to see your newly deployed model in the *Models* portion of the AI Platform section of the GCP console.

In [ ]:
%%bash
# Set necessary variables
MODEL_NAME = "babyweight"
MODEL_VERSION = "ml_on_gcp"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/babyweight/export/exporter/ | tail -1)

# Set the region to global by executing the following command
gcloud config set ai_platform/region global

echo "Deploying the model '$MODEL_NAME', version '$MODEL_VERSION' from $MODEL_LOCATION"
echo "... this will take a few minutes"

# Deploy trained model
gcloud ai-platform models create ${MODEL_NAME} --regions $REGION
# Create new AI Platform version
gcloud ai-platform versions create ${MODEL_VERSION} \
    --model ${MODEL_NAME} \
    --origin ${MODEL_LOCATION} \
    --runtime-version $TFVERSION